In [12]:
from BlackBox.lmart.LambdaMart_utils import LMARTGridsearch
import pandas as pd
import numpy as np
%reload_ext autoreload
%autoreload 2

In [2]:
lMart_parameter = dict(
    boosting_type=["dart", "gbdt"],
    num_leaves=[2, 5, 10, 20, 30, 40],
    max_depth=[-1],
    n_estimators=[75, 80, 100, 150, 200],
    learning_rate=[0.02, 0.05, 0.08, 0.1, 0.15, 0.2],
    reg_lambda=[0.00005, 0.0001, 0.0002, 0.0003]
)

# Grid-search

In [3]:
gs = LMARTGridsearch(path_dataset="../../outputs/scores.csv", random_state=841)

In [ ]:
best_ = gs.grid_search(lMart_parameter)
gs.save_model(best_[0],"lgbmranker_unbalanced")

# Metric evaluation

In [5]:
best_model = gs.load_model("lgbmranker_unbalanced")

In [7]:
# nDCG
nDCG_train = gs.eval_model(model=best_model, df=gs.train, qIds=gs.qIds_train, nDCG_at=[1,10,15])
nDCG_valid = gs.eval_model(model=best_model, df=gs.valid, qIds=gs.qIds_val, nDCG_at=[1,10,15])
nDCG_test = gs.eval_model(model=best_model, df=gs.test, qIds=gs.qIds_test, nDCG_at=[1,10,15])

other_evals_tr = gs.other_eval(df=gs.test, qIds=gs.qIds_test, nDCG_at=[1,10,15])

display(pd.DataFrame([nDCG_train,nDCG_valid,nDCG_test],index=["Training","Validation","Test"]))

print("Test set")
display(pd.DataFrame(other_evals_tr).T)

,nDCG@1,nDCG@10,nDCG@15
Training,0.9267,0.8225,0.8283
Validation,0.7908,0.7971,0.8296
Test,0.7933,0.7701,0.7957


Test set


,nDCG@1,nDCG@10,nDCG@15
random_permutation,0.5636,0.6369,0.6784
perfect_nDCG,1.0000,1.0000,1.0000
worste_nDCG,0.2832,0.3745,0.4458


# Example of Job-offer

In [11]:
gs.test[gs.test["qId"]==15][["kId","labels"]].head(15)

,kId,labels
3006,6,2
3008,8,2
3014,14,2
3016,16,1
3020,20,2
3024,24,2
3026,26,2
3027,27,2
3029,29,1
3032,32,1


In [13]:
features = gs.test[gs.test["qId"]==15]
y_pred = best_model.predict(np.asarray(features.iloc[:,2:13].values))
y_pred = pd.DataFrame(y_pred, index=features.index, columns=["lambdas"])
dt_final = pd.merge(features, y_pred, left_index=True, right_index=True)
dt_final.sort_values("lambdas",ascending=False)[["kId","labels"]].head(15)

,kId,labels
3165,165,4
3167,167,2
3051,51,2
3187,187,2
3103,103,3
3027,27,2
3197,197,2
3097,97,2
3171,171,2
3199,199,3


In [ ]:
# qId= 1
# job_curricula = gs.test[gs.test["qId"] == qId]
# 
# y_pred = best_model.predict(job_curricula.iloc[:,2:13])
# 
# y_pred = pd.DataFrame(y_pred, index=job_curricula.index, columns=["lambdas"])
# dt_final = pd.merge(job_curricula, y_pred, left_index=True, right_index=True)
# dt_final.sort_values("labels",ascending=False)["labels"].head(15)
# dt_final.sort_values("lambdas",ascending=False)["labels"].head(15)

# Grid search - balanced

In [29]:
gs = LMARTGridsearch(path_dataset="../../outputs/balanced_scores.csv", random_state=841)

In [30]:
best_ = gs.grid_search(lMart_parameter)
gs.save_model(best_[0],"lgbmranker_balanced")

100%|██████████| 1440/1440 [06:12<00:00,  3.86it/s, nDCG_15=0.935]


In [31]:
best_model = gs.load_model("lgbmranker_unbalanced")

In [32]:
# nDCG
nDCG_train = gs.eval_model(model=best_model, df=gs.train, qIds=gs.qIds_train, nDCG_at=[1,10,15])
nDCG_valid = gs.eval_model(model=best_model, df=gs.valid, qIds=gs.qIds_val, nDCG_at=[1,10,15])
nDCG_test = gs.eval_model(model=best_model, df=gs.test, qIds=gs.qIds_test, nDCG_at=[1,10,15])

other_evals_tr = gs.other_eval(df=gs.test, qIds=gs.qIds_test, nDCG_at=[1,10,15])

display(pd.DataFrame([nDCG_train,nDCG_valid,nDCG_test],index=["Training","Validation","Test"]))

print("Test set")
display(pd.DataFrame(other_evals_tr).T)

,nDCG@1,nDCG@10,nDCG@15
Training,0.8989,0.8713,0.9038
Validation,0.8900,0.9358,0.9457
Test,0.8550,0.8885,0.9264


Test set


,nDCG@1,nDCG@10,nDCG@15
random_permutation,0.6313,0.7943,0.8548
perfect_nDCG,1.0000,1.0000,1.0000
worste_nDCG,0.3243,0.6195,0.7346
